In [1]:
import re
import os
import io
import math
import boto3
import pandas as pd
from trp import Document

In [2]:
# Document
documentName = "assignment.png"
REGION = "us-east-2"

In [3]:
# Amazon Textract client
textract = boto3.client('textract',REGION)

In [4]:
with open("assignment.png", 'rb') as image:
    response = textract.analyze_document(Document={'Bytes': image.read()},FeatureTypes=['TABLES'])

In [5]:
# get the doc object for response_json
doc = Document(response)

### Utility Functions

In [6]:
def get_page_table(doc):
    """
    This method will give all tabular data in given page.
    """
    page_table_dict = {}
    for page_cnt,page in enumerate(doc.pages,start=1):
        page_table_dict[page_cnt] = []
        for table in page.tables:
            temp_table = []
            for r, row in enumerate(table.rows):
                temp_row = []
                for c, cell in enumerate(row.cells):
                    temp_row.append(cell.text)
                # add ech row to table
                temp_table.append(temp_row)
        
            # add table in page to dict
            page_table_dict[page_cnt].append(temp_table)
        
    
    return page_table_dict
                    
          
def get_table_df(page_table_dict):
    table_df = {}
    for page_cnt,tables in page_table_dict.items():
        table_df[page_cnt] = []
        for table in tables:
            df = pd.DataFrame(table)
            if not df.empty:
                table_df[page_cnt].append(df)
    return table_df

def GetFileName(filename,extension = ".xlsx"):
    name = os.path.basename(filename)
    f_name = name.split(".")[0] + extension
    return f_name

# save df to excel for better view
def saveExcel(df_dict,filname,spaces = 1,single_sheet = True):
    
    output_path = GetFileName(filname)
    writer = pd.ExcelWriter(output_path,engine='xlsxwriter')
    
    for page_cnt,tables in df_dict.items():
        for table in tables:
            # clean the df
            table= table.replace("\n"," ",regex=True)
            table= table.replace(" +"," ",regex=True)
            table.to_excel(writer,"page_{}".format(page_cnt) , startcol = 0 , startrow = 0)
        
    writer.save()

In [7]:
page_table_dict = get_page_table(doc)
table_df = get_table_df(page_table_dict)

In [8]:
tables = table_df[1]

In [9]:
print("Total tables found in given document :- {}".format(len(tables)))

Total tables found in given document :- 2


#### Table_0

In [10]:
table = tables[0]
table= table.replace("\n"," ",regex=True)
table= table.replace(" +"," ",regex=True)

In [11]:
table

,0,1,2
0,4:Review the information on file for. accuracy...,box.,
1,,ON FILE,UPDATES
2,1 How many motorized vehicles licensed for roa...,2,2
3,2. How many residential properties are owned o...,1,0
4,"3. How many watercraft, OTHER than canoes, Jet...",0,O
5,"4. How many Jet Skis, Waverunners or other per...",0,0
6,5. What is the total number of drivers? (Inclu...,2,2
7,6. How many drivers are under the age of 22? (...,0,O
8,7. How many drivers are age 70 or over? (Inclu...,1,I
9,8. How many total moving violations have all d...,0,0


#### Table_1

In [12]:
table = tables[1]
table= table.replace("\n"," ",regex=True)
table= table.replace(" +"," ",regex=True)
table

,0,1
0,CHECK,"IF ""YES"""
1,15. Have you or any other driver had an arrest...,
2,16 Have you or any member of your household be...,
3,17. Do you or any member of your household hav...,
4,18. Has any one driver had more than 3 moving ...,


#### Save tables to excel for better view

In [13]:
saveExcel(table_df,documentName)

### Questionw With Answer Dictionary:

In [14]:
table = tables[0]
table= table.replace("\n"," ",regex=True)
table= table.replace(" +"," ",regex=True)
table.columns = ["Questions","Actual_Ans","Written_Ans"]

In [15]:
# questions are present in 0th colomn
questions = table["Questions"].values
# written ans in 1st column
written_ans = table["Written_Ans"]

#### It it found that 0 is bieng detected as 'o' in many handwritten cases

In [16]:
# it it found that 0 is bieng detected as o in many handwritten cases
written_ans = written_ans.replace(r"(?i)o",'0',regex=True,)
written_ans

0                     
1             UPDATES 
2                   2 
3                   0 
4                   0 
5                   0 
6                   2 
7                   0 
8                   I 
9                   0 
10                  0 
11                  0 
12                  0 
13                    
14    N0T_SELECTED, 0 
15                  0 
16                  0 
Name: Written_Ans, dtype: object

In [23]:
question_answer = {}
for question,answer in zip(questions,written_ans):
    # filtering out noisy answer
    is_number = re.findall(r"\d+",answer,flags=re.IGNORECASE)
    if is_number:
        question_answer[question] = answer
        print("++++++++++++[Questions & Ans]++++++++++++++")
        print(question)
        print(answer)

++++++++++++[Questions & Ans]++++++++++++++
1 How many motorized vehicles licensed for road use (i.e., motorhomes, motorcycles, cars, etc.) are owned (titled or registered to), leased, rented, or regularly operated by you or any member of your household? (See page 3 for details. Do not count antique, classic, or collectible vehicles - see question 10 ) 
2 
++++++++++++[Questions & Ans]++++++++++++++
2. How many residential properties are owned or rented by you or any member of your household? 1-4 family units are eligible and should be counted as one property. Do not include residential properties that are covered under a Commercial General Liability Policy or other non-personal Premises Liability Policy because they are excluded from coverage. (See page 3 for details.) 
0 
++++++++++++[Questions & Ans]++++++++++++++
3. How many watercraft, OTHER than canoes, Jet Skis, Waverunners or other personal watercraft, are owned or regularty operated by you or any member of your household? (See